## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Código para quem não tem endereço de email do Inteli, só colocar o link e id do arquivo

#import gdown
#arquivo_destino_colab = "dataset.csv"

# https://drive.google.com/drive/folders/11gFZUmhu6Arzk_IqqOD9lRX9Re6bIzVL?usp=sharing
#doc_id = "11gFZUmhu6Arzk_IqqOD9lRX9Re6bIzVL"

#URL = f"https://drive.google.com/uc?id={doc_id}"

#gdown.download(URL, arquivo_destino_colab, quiet=False)

#df = pd.read_csv(f"/content/{arquivo_destino_colab}", sep=";")

#df.head()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importação das Tabelas de 2023 e 2024

In [4]:
ga_universal_2019a2022 = pd.read_csv('/content/drive/Shareddrives/M10_SI/ga_universal_2019a2022.csv')
acesso_eventos_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/acessos_eventos_2023.csv')
acessos_fontes_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/acessos_fontes_2024.csv')
age_sessions_big_numbers_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/age_sessions_big_numbers_2024.csv')
hour_users_big_numbers_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/hour_users_big_numbers_2024.csv')
week_users_big_numbers_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/week_users_big_numbers_2024.csv')
google_ads_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/google_ads_2023.csv')
country_city_views_big_numbers_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/country_city_views_big_numbers_2023.csv')
country_city_views_big_numbers_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/country_city_views_big_numbers_2024.csv')
ga_universal_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/ga_universal_2023.csv')
acessos_fontes_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/acessos_fontes_2023.csv')
age_sessions_big_numbers_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/age_sessions_big_numbers_2023.csv')
hour_users_big_numbers_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/hour_users_big_numbers_2023.csv')
week_users_big_numbers_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/week_users_big_numbers_2023.csv')
acessos_geral_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/acessos_geral_2023.csv')
# acessos_geral_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/acessos_geral_2024.csv')
users_country_big_numbers_geo_2023 = pd.read_csv('/content/drive/Shareddrives/M10_SI/users_country_big_numbers_geo_2023.csv')
users_country_big_numbers_geo_2024 = pd.read_csv('/content/drive/Shareddrives/M10_SI/users_country_big_numbers_geo_2024.csv')

## Correlação tempo e semana com monte Monte Carlo

### O algoritmo desenvolvido tem como propósito a realização de uma análise estatística detalhada sobre distribuições de probabilidade de eventos em um conjunto de dados, utilizando o método de simulação de Monte Carlo. Este método é útil para modelar fenômenos com grande número de variáveis e calcular probabilidades complexas.

##### **Funções Auxiliares**
Função calcular_prob: Calcula a distribuição de probabilidade para uma coluna específica de um DataFrame.

In [5]:
def calcular_prob(df, index, value):
  '''
  Função calcular_prob: Calcula a distribuição de probabilidade para uma coluna específica de um DataFrame.
  '''
  res = {}
  df_dict = df.set_index(index).to_dict()[value]
  total = sum(df_dict.values())
  for chave, valor in df_dict.items():
    res[chave] = valor/total


  if sum(res.values()) != 1:
    res["margem de erro"] = 1 - sum(res.values())

  return res

##### **Classes**

Classe Camada: Representa uma camada de decisão dentro do modelo de Monte Carlo. Cada camada possui uma lista de probabilidades e uma lista de alternativas correspondentes.

Classe GaltonBoard: Utiliza múltiplas camadas para simular trajetórias através de um processo estocástico, reproduzindo a aleatoriedade do fenômeno estudado.

In [6]:
import numpy as np
from typing import List


class Camada:
    def __init__(self, probs: list, alternativas: list):
        self.probs = probs
        self.alternativas = list(map(str, alternativas))

        if sum(probs) != 1:
            raise Exception("A soma das probabilidades deve resultar em 1")

        if len(probs) != len(alternativas):
            raise Exception("Deve conter a mesma quantidade probabilidades e alternativas")


class GaltonBoard:
    def __init__(self):
        self.camadas = []

    def adicionar_camadas(self, camadas: List[Camada]):
        self.camadas.extend(camadas)

    def simular_montecarlo(self, num_iteracoes):
        if len(self.camadas) == 0:
            raise Exception("é necessário adicionar uma camada para fazer a simulação")

        todas_escolhas = {}
        for _ in range(num_iteracoes):
            caminho_percorrido = []
            for camada in self.camadas:
                ramificacao_escolida = np.random.choice(camada.alternativas, p= camada.probs)
                caminho_percorrido.append(ramificacao_escolida)

            caminho_percorrido_str = " - ".join(caminho_percorrido)

            if caminho_percorrido_str not in todas_escolhas.keys():
                todas_escolhas[caminho_percorrido_str] = 1

            else:
                todas_escolhas[caminho_percorrido_str] += 1

        return todas_escolhas


##### **Utilização**

Para utilizar o algoritmo, primeiramente são calculadas as probabilidades para diversas variáveis (como dia da semana, hora, idade e cidade) e, em seguida, essas probabilidades são usadas para configurar as camadas de decisão do modelo de Monte Carlo. A simulação é então executada para um número específico de iterações, produzindo um conjunto de possíveis resultados e suas frequências. Esta abordagem é especialmente útil para entender padrões complexos e simular cenários futuros baseados em dados históricos.

In [7]:
probs_dia = calcular_prob(week_users_big_numbers_2023, "Day of week", "Active users")
probs_hora = calcular_prob(hour_users_big_numbers_2023, "Hour", "Active users")
probs_idade = calcular_prob(age_sessions_big_numbers_2023, "Age", "Sessions" )
probs_cidades = calcular_prob(country_city_views_big_numbers_2023, "City", "Views")

In [9]:
dia = Camada(alternativas = list(probs_dia.keys()), probs = list(probs_dia.values()))
hora = Camada(alternativas = list(probs_hora.keys()), probs = list(probs_hora.values()))
idade = Camada(alternativas = list(probs_idade.keys()), probs = list(probs_idade.values()))
cidade = Camada(alternativas = list(probs_cidades.keys()), probs = list(probs_cidades.values()))

In [34]:
x = GaltonBoard()

In [35]:
x.adicionar_camadas([dia, hora, idade, cidade])

In [36]:
resultados = x.simular_montecarlo(100000)

In [37]:
def obter_maiores_probs(resultado_da_simulacao: dict, quantidade):
  valores = []
  for chave, valor in resultado_da_simulacao.items():
    valores.append((chave, valor))

  maiores_valores = sorted(valores, key= lambda x: x[1], reverse = True)[:quantidade]

  return maiores_valores

In [48]:
obter_maiores_probs(resultados, 10)

[('4 - 20 - unknown - Sao Paulo', 107),
 ('6 - 21 - unknown - Sao Paulo', 97),
 ('3 - 19 - unknown - Sao Paulo', 87),
 ('3 - 20 - unknown - Sao Paulo', 86),
 ('2 - 20 - unknown - Sao Paulo', 86),
 ('6 - 20 - unknown - Sao Paulo', 85),
 ('5 - 20 - unknown - Sao Paulo', 84),
 ('3 - 18 - unknown - Sao Paulo', 84),
 ('1 - 22 - unknown - Sao Paulo', 82),
 ('5 - 19 - unknown - Sao Paulo', 80)]

In [17]:
def obter_manores_probs(probConjunta: dict, quantidade):
  valores = []
  for chave, valor in probConjunta.items():
    valores.append((chave, valor))

  maiores_valores = sorted(valores, key= lambda x: x[1])[:quantidade]

  return maiores_valores

In [18]:
obter_manores_probs(resultados, 10)

[('6 - 12 - unknown - Guaiba', 1),
 ('3 - 7 - unknown - Maringa', 1),
 ('6 - 8 - unknown - Uruguaiana', 1),
 ('6 - 6 - unknown - Parobe', 1),
 ('5 - 16 - 35-44 - Lisbon', 1),
 ('4 - 16 - 25-34 - Sao Bernardo do Campo', 1),
 ('1 - 23 - 25-34 - Petrolina', 1),
 ('5 - 22 - unknown - Maraba', 1),
 ('2 - 15 - 35-44 - Fortaleza', 1),
 ('1 - 15 - unknown - Sananduva', 1)]